In [6]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
import scipy
from scipy.sparse import lil_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader

from az_utils import *
from ember_model import *




def get_year_data(data_dir, year, train=True):
    
    if train:
        data_dir = data_dir + '/'
        XY_train = np.load(data_dir + str(year) + '_Domain_AZ_Train_Transformed.npz', allow_pickle=True)
        X_tr, Y_tr = XY_train['X_train'], XY_train['Y_train']
        
        return X_tr, Y_tr
    else:
        data_dir = data_dir + '/'
        XY_test = np.load(data_dir + str(year) + '_Domain_AZ_Test_Transformed.npz', allow_pickle=True)
        X_test, Y_test = XY_test['X_test'], XY_test['Y_test']

        return X_test, Y_test 


def get_MemoryData(X, Y, memory_budget):
    indx = [i for i in range(len(Y))]
    random.shuffle(indx)
    
    replay_index = indx[:memory_budget]
    X_train = X[replay_index]
    Y_train = Y[replay_index]
    
    return X_train, Y_train

def create_empty_matrix(m, n):
    return np.zeros((m, n))

In [7]:
def get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=True, none=False):
    
    if train:
        if none:
            X_tr, Y_tr = get_year_data(data_dir, task_years[-1])
            X_train, Y_train  = X_tr, Y_tr
            print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
            return X_train, Y_train
        
        else:
            X_tr, Y_tr = get_year_data(data_dir, task_years[-1])
            print(f'Current Task Year {task_years[-1]} data X {X_tr.shape} Y {Y_tr.shape}')

            if len(task_years) != 1:
                previous_Xs, previous_Ys = [], []
                for year in task_years[:-1]:
                    pre_X_tr, pre_Y_tr = get_year_data(data_dir, year)


                    pre_X_tr, pre_Y_tr = np.array(pre_X_tr), np.array(pre_Y_tr)
                    #print(f'pre_X_tr {pre_X_tr.shape}  pre_Y_tr {pre_Y_tr.shape}')

                    for idx, prevSample in enumerate(pre_X_tr):
                        previous_Xs.append(prevSample)
                        previous_Ys.append(pre_Y_tr[idx])


                previous_Xs, previous_Ys = np.array(previous_Xs), np.array(previous_Ys)  

                #print(f'Y_tr {Y_tr.shape}  previous_Ys {previous_Ys.shape}')
                if joint:
                    X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                else:
                    if memory_budget >= len(previous_Ys):
                        X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                    else:
                        previous_Xs, previous_Ys = get_MemoryData(previous_Xs, previous_Ys, memory_budget)
                        X_tr, Y_tr = np.concatenate((X_tr, previous_Xs)), np.concatenate((Y_tr, previous_Ys))
                        #print(f'memory_budget {memory_budget}  Y_tr {previous_Ys.shape} ')
                        assert memory_budget == len(previous_Ys)


            X_train, Y_train  = X_tr, Y_tr
            print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
            return X_train, Y_train
    else:
        X_te, Y_te = get_year_data(data_dir, task_years[-1], train=False)
        for year in task_years[:-1]:
            pre_X_te, pre_Y_te = get_year_data(data_dir, year, train=False)
            
            X_te, Y_te = np.concatenate((X_te, pre_X_te)), np.concatenate((Y_te, pre_Y_te))

        X_test, Y_test  = X_te, Y_te
        print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
        return X_test, Y_test

In [77]:
def testing_aucscore(model, X_test, Y_test, batch_size, device):
    #X_te = torch.from_numpy(X_test).type(torch.FloatTensor)
    #y_te = torch.from_numpy(Y_test).type(torch.FloatTensor) 
    
    testloader = get_dataloader(X_test, Y_test, batch_size, train_data=False)   
    
    model.eval()
    y_pred_list = []
    y_true_list = []
    test_acc = []
    with torch.no_grad():
        for x_batch, y_batch in tqdm(testloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            tmp_test_acc = binary_acc(y_test_pred, y_batch)
            test_acc.append(tmp_test_acc.item())
            
            y_pred_tag = torch.round(y_test_pred).squeeze(1)
            y_pred_list += list(y_pred_tag.cpu().numpy())
            y_true_list += list(y_batch.cpu().numpy())
        
            
    #correct_test_results = (np.array(y_pred_list) == np.array(y_true_list)).sum()
    #acc = correct_test_results/len(y_true_list)
    
#     from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
    
#     correct_labels, predicted_labels = np.array(y_true_list), np.array(y_pred_list)
    
#     roc_auc = roc_auc_score(correct_labels, predicted_labels)
#     precision = precision_score(correct_labels, predicted_labels, average='micro')
#     recall = recall_score(correct_labels, predicted_labels, average='micro')
#     f1score = f1_score(correct_labels, predicted_labels, average='macro')
    
    #print(f'test accuracy {np.mean(test_acc)} and ROC-AUC {roc_auc}')

    #wrong_good, wrong_mal, top_k_mistaken_families = \
    #            get_mistaken_stats(np.array(y_true_list), np.array(np.round(y_pred_list)), Y_test_family, top_k)
    
    #return np.mean(test_acc), roc_auc, precision, recall, f1score
    return np.mean(test_acc)

# parser = argparse.ArgumentParser()
# parser.add_argument('--num_exps', type=int, default=1, required=False, help='Number of Experiments to Run.')
# parser.add_argument('--num_epoch', type=int, default=100, required=False)
# parser.add_argument('--batch_size', type=int, default=4096, required=False)
# parser.add_argument('--grs_joint',  action="store_true", required=False)
# parser.add_argument('--grs_none',  action="store_true", required=False)
# parser.add_argument('--memory_budget', type=int, required=False)
# parser.add_argument('--data_dir', type=str,\
#                     default='/home/mr6564/continual_research/AZ_Data/Domain_Transformed/', required=False)

# args = parser.parse_args()

# data_dir = args.data_dir
# num_exps = args.num_exps
# num_epoch = args.num_epoch
# batch_size = args.batch_size
# memory_budget = args.memory_budget

data_dir = '/home/mr6564/continual_research/AZ_Data/Domain_Transformed/'
num_exps = 2
num_epoch = 200
batch_size = 4096
memory_budget = 'none'

# if args.grs_joint:
#     memory_budget = 'joint'
    
# if args.grs_none:
#     memory_budget = 'none'


patience = 5

exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]

all_task_years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']


input_features = 1789

replay_type, current_task = 'azdomain', 'azdomain'


num_tasks = len(all_task_years)
results_matrix = create_empty_matrix(num_tasks, num_tasks)
#print(matrix)


global_acc_joint = []
global_future_acc = []
cnt =  1    
for exp in exp_seeds:
    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net(input_features)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    
    
    
#     standardization = StandardScaler()
#     standard_scaler = None
    for task_year in range(len(all_task_years)):
                
        print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Round {cnt} ...')
        task_start = time.time()
        current_task = all_task_years[task_year]
        task_years = all_task_years[:task_year+1]
        print(f'Current Task {current_task} with Budget {memory_budget}')


        model_save_dir = '../az_model_cd/GRS_SavedModel' + '/GRSModel_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(model_save_dir)
        
        opt_save_path = '../az_model_cd/GRS_SavedModel' + '/GRSOpt_' + str(memory_budget) + '/' + str(current_task) + '/'
        create_parent_folder(opt_save_path)
        
        results_save_dir =  '../az_model_cd/GRS_SavedResults_' +'/GRS_' + str(memory_budget) + '/' 
        create_parent_folder(results_save_dir)
        
        
#         if args.grs_joint:
#             X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=True)
#             # X_test, Y_test = get_GRS_data(data_dir, task_years, memory_budget, train=False, joint=True)
#         elif args.grs_none:
#             X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, none=True)
#             # X_test, Y_test = get_GRS_data(data_dir, task_years, memory_budget, train=False, none=True)
#         else:
#             X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, joint=False)
#             # X_test, Y_test = get_GRS_data(data_dir, task_years, memory_budget, train=False, joint=False)
        
        X_train, Y_train = get_GRS_data(data_dir, task_years, memory_budget, train=True, none=True)
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
        
#         if args.grs_joint:
#                 standardization = StandardScaler()
#                 standard_scaler = None
#                 standard_scaler = standardization.fit(X_train)
#         else:        
#                 standard_scaler = standardization.partial_fit(X_train)

#         X_train = standard_scaler.transform(X_train)
#         X_test = standard_scaler.transform(X_test)

        from sklearn.model_selection import train_test_split
        X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
        X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)
        # X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)        
        
        print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Training ...')
        
        
        
        task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                                     model, model_save_dir, opt_save_path, X_train, Y_train,\
                                     X_valid, Y_valid, patience, batch_size, device, optimizer, num_epoch,\
                                     criterion, replay_type, current_task, exp, earlystopping=True)


        end_time = time.time()
        print(f'Elapsed time {(end_time - start_time)/60} mins.') 


        best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
        print(f'loading best model {best_model_path}')
        model.load_state_dict(torch.load(best_model_path))

        #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
        best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
        print(f'loading best optimizer {best_optimizer}')
        optimizer.load_state_dict(torch.load(best_optimizer))

        
        for t in range(num_tasks):
            y_t = all_task_years[t]
            X_te, Y_te = get_year_data(data_dir, y_t, train=False)
            
            # acc, _, _, _, _ = testing_aucscore(model, X_te, Y_te, batch_size, device)
            acc = testing_aucscore(model, X_te, Y_te, batch_size, device)
            
            results_matrix[task_year][t] = acc
            
#             if results_matrix[task_year][t] == 0.0:
#                 results_matrix[task_year][t] = acc
#             else:
#                 tmp_acc = results_matrix[task_year][t]
#                 results_matrix[task_year][t] = (np.sum(tmp_acc,acc)/2)
            
        #print(results_matrix)
                
            
        end_time = time.time()
        print(f'Elapsed time {(end_time - start_time)/60} mins.')    
        
       
#         results_f = open(os.path.join('./Submission_Domain_Concept_Drift/' + 'grs_' + str(memory_budget) + '_results.txt'), 'a')
#         result_string = '{}\t{}\t{}\t{}\t{}\t\n'.format(current_task, acc, precision, recall, f1score)
        
#         results_f.write(result_string)
#         results_f.flush()
#         results_f.close()
    
    joint_acc_all = []
    future_acc_all = []
    for t in range(num_tasks):
        #print(f'res \t {results_matrix[t]}\n')
    #     if t == first_task:
    #         print(f'task {t}\t{results_matrix[t][0]}\n')
    #         curr_task += 1
    #     else:
    #         print(f'res \t {results_matrix[t]}\n')

        joint_acc = np.mean(results_matrix[t][0:t+1])
        future_acc = np.mean(results_matrix[t][t+1:])
        #print(f'task {t+1}\t{results_matrix[t][t+1:]}\n')
        joint_acc_all.append(joint_acc)

        if t+1 != num_tasks:
            #print(future_acc)
            future_acc_all.append(future_acc)

    #     print(f'task {t+1}\tAvg-Acc-Till-Current-Task\t{joint_acc}\t\tFuture-Acc\t{future_acc}')

    print(f'Avg-Acc-Till-Current-Task-Joint\t{np.mean(joint_acc_all)}\t\tFuture-Acc\t{np.mean(future_acc_all)}')
    
    global_acc_joint.append(np.mean(joint_acc_all))
    global_future_acc.append(np.mean(future_acc_all))
    
    results_f = open(os.path.join('./Submission_Domain_Concept_Drift/' + 'grs_' + str(memory_budget) + '_results.txt'), 'a')
    result_string = '{}\t{}\t\n'.format(np.mean(joint_acc_all), np.mean(future_acc_all))

    results_f.write(result_string)
    results_f.flush()
    results_f.close()
    
    
    end_time = time.time()
    cnt += 1
    print(f'Elapsed time {(end_time - start_time)/60} mins.')
    
    del model_save_dir
    del opt_save_path
    del results_save_dir

print(f'Global-Acc-Joint\t{np.mean(global_acc_joint)}\t\tGlobal-Future-Acc\t{np.mean(global_future_acc)}')

Torch 1.2.0+cu92 CUDA 9.2.148
Model has 2.525953m parameters

2024-05-26 22:50:21 Round 1 ...
Current Task 2008 with Budget none
X_train (36333, 1789) Y_train (36333,)

2024-05-26 22:50:22 Standardizing ...


  0%|          | 0/7 [00:00<?, ?it/s]

2024-05-26 22:50:22 Training ...
Epoch 1 of 200


  0%|          | 0/7 [00:00<?, ?it/s]

Train Loss: 0.3276, Train Acc: 0.8826
Val Loss: 0.4664, Val Acc: 0.9744
Validation loss decreased (inf --> 0.466356).  Saving model ...
../az_model_cd/GRS_SavedModel/GRSModel_none/2008/best_model_epoch_1.pt
../az_model_cd/GRS_SavedModel/GRSOpt_none/2008/best_optimizer_epoch_1.pt
Epoch 2 of 200


...

  0%|          | 0/1 [00:00<?, ?it/s]

Train Loss: 0.0214, Train Acc: 0.9934
Val Loss: 0.0533, Val Acc: 0.9877
EarlyStopping counter: 5 out of 5
Early stopping
Training time: 0.663 minutes
Elapsed time 5.139290448029836 mins.
loading best model ../az_model_cd/GRS_SavedModel/GRSModel_none/2016/best_model_epoch_16.pt
loading best optimizer ../az_model_cd/GRS_SavedModel/GRSOpt_none/2016/best_optimizer_epoch_16.pt


100%|██████████| 3/3 [00:00<00:00,  6.58it/s]

Elapsed time 5.22671416203181 mins.
Avg-Acc-Till-Current-Task-Joint	0.8978964356842424		Future-Acc	0.7768589081124417


FileNotFoundError: [Errno 2] No such file or directory: './Submission_Domain_Concept_Drift/grs_none_results.txt'

In [78]:

joint_acc_all = []
future_acc_all = []
for t in range(num_tasks):
    #print(f'res \t {results_matrix[t]}\n')
#     if t == first_task:
#         print(f'task {t}\t{results_matrix[t][0]}\n')
#         curr_task += 1
#     else:
#         print(f'res \t {results_matrix[t]}\n')
    
    joint_acc = np.mean(results_matrix[t][0:t+1])
    future_acc = np.mean(results_matrix[t][t+1:])
    #print(f'task {t+1}\t{results_matrix[t][t+1:]}\n')
    joint_acc_all.append(joint_acc)
    
    if t+1 != num_tasks:
        #print(future_acc)
        future_acc_all.append(future_acc)
    
#     print(f'task {t+1}\tAvg-Acc-Till-Current-Task\t{joint_acc}\t\tFuture-Acc\t{future_acc}')

print(f'Avg-Acc-Till-Current-Task-Joint\t{np.mean(joint_acc_all)}\t\tFuture-Acc\t{np.mean(future_acc_all)}')

Avg-Acc-Till-Current-Task-Joint	0.8978964356842424		Future-Acc	0.7768589081124417


In [ ]:
# None
Avg-Acc-Till-Current-Task-Joint	0.8978964356842424		Future-Acc	0.7768589081124417

# Joint
Avg-Acc-Till-Current-Task-Joint	0.9634991586357268		Future-Acc	0.7949101760687809
Avg-Acc-Till-Current-Task-Joint	0.9636760994301411		Future-Acc	0.802318911295798

In [54]:
num_tasks = len(all_task_years)
results_matrix = create_empty_matrix(num_tasks, num_tasks)
#print(matrix)

In [58]:
results_matrix[0][0] == 0

True